# Circuit Tutorial - QuantumCircuitsMPS.jl

This tutorial demonstrates the Circuit API workflow for building, visualizing, and simulating quantum circuits in QuantumCircuitsMPS.jl. The Circuit API provides a lazy/symbolic representation that separates circuit construction from execution, enabling visualization and inspection before running expensive simulations.

## Key Concepts

1. **Build**: Circuit created with do-block syntax (lazy/symbolic)
2. **Visualize**: SVG diagrams without execution
3. **Simulate**: Deterministic execution with explicit RNG seeds

## Why Use the Circuit API?

- ✓ Inspect circuit structure before expensive simulation
- ✓ Reuse same circuit with different initial states
- ✓ Export circuit diagrams for documentation/papers
- ✓ Separate logical structure from execution details

## Setup

First, activate the project environment and import QuantumCircuitsMPS:

In [1]:
using Pkg; Pkg.activate(dirname(@__DIR__))
using QuantumCircuitsMPS

  Activating project at `/mnt/d/Rutgers/QuantumCircuitsMPS.jl`


## Section 1: Setup and Parameters

Define the system parameters for our quantum circuit:

In [2]:
# Define system parameters
const L = 4                    # System size (number of qubits)
const bc = :periodic           # Boundary conditions (:periodic or :open)
const n_steps = 50             # Number of circuit timesteps
const p_reset = 0.3            # Probability of reset operation

println("=" ^ 70)
println("Circuit Tutorial - QuantumCircuitsMPS.jl")
println("=" ^ 70)
println()
println("Parameters:")
println("  L = $L (system size)")
println("  bc = $bc (boundary conditions)")
println("  n_steps = $n_steps (circuit timesteps)")
println("  p_reset = $p_reset (reset probability)")
println()

Circuit Tutorial - QuantumCircuitsMPS.jl

Parameters:
  L = 4 (system size)
  bc = periodic (boundary conditions)
  n_steps = 50 (circuit timesteps)
  p_reset = 0.3 (reset probability)



## Section 2: Building Circuits with Do-Block Syntax

The Circuit API uses a lazy/symbolic representation. When you build a circuit, no quantum operations are executed - instead, the circuit structure is recorded as a data structure that can be inspected, visualized, and executed later.

The do-block syntax provides a clean way to define circuit operations:
- `apply!(c, gate, geometry)` - for deterministic gates
- `apply_with_prob!(c; rng=:ctrl, outcomes=[...])` - for stochastic operations

In [3]:
println("Building quantum circuit...")
println()

# Build circuit with stochastic operations
# This circuit models a measurement-reset protocol:
#   - With probability p_reset: Reset qubit to |0⟩ state
#   - With probability 1-p_reset: Apply random unitary (HaarRandom)
#
# The StaircaseRight(1) geometry applies operations in a staircase pattern,
# moving one site to the right each timestep (with periodic wrapping)
circuit = Circuit(L=L, bc=bc, n_steps=n_steps) do c
    apply_with_prob!(c; rng=:ctrl, outcomes=[
        (probability=p_reset, gate=Reset(), geometry=StaircaseLeft(1)),
        (probability=1-p_reset, gate=HaarRandom(), geometry=StaircaseRight(1))
    ])
end

println("✓ Circuit built successfully")
println("  Total timesteps: $(circuit.n_steps)")
println("  System size: $(circuit.L) qubits")
println("  Boundary conditions: $(circuit.bc)")
println()

Building quantum circuit...

✓ Circuit built successfully
  Total timesteps: 50
  System size: 4 qubits
  Boundary conditions: periodic


  Total timesteps: 50
  System size: 4 qubits
  Boundary conditions: periodic



## Section 3: Adding Deterministic Gates

In addition to stochastic operations, you can add deterministic gates using `apply!()`. This example builds a circuit with both types of operations:

In [4]:
println("Building circuit with mixed gate types...")
println()

mixed_circuit = Circuit(L=L, bc=:open, n_steps=20) do c
    # Apply PauliX gate to site 1
    apply!(c, PauliX(), SingleSite(1))
    
    # Stochastic operation: probabilistic reset
    apply_with_prob!(c; rng=:ctrl, outcomes=[
        (probability=0.5, gate=Reset(), geometry=StaircaseLeft(1)),
        (probability=0.5, gate=HaarRandom(), geometry=StaircaseRight(1))
    ])
    
    # Apply PauliZ gate to last site
    apply!(c, PauliZ(), SingleSite(L))
end

println("✓ Mixed-gate circuit built")
println("  Operations include: PauliX, Reset, HaarRandom, PauliZ")
println()

Building circuit with mixed gate types...

✓ Mixed-gate circuit built
  Operations include: PauliX, Reset, HaarRandom, PauliZ



## Section 4: SVG Visualization (Optional)

The `plot_circuit` function generates publication-quality SVG diagrams. This requires Luxor.jl as a weak dependency (optional).

We use try-catch to gracefully handle the case where Luxor is not installed:

In [5]:
println("SVG Visualization (optional - requires Luxor.jl):")
println()

# Ensure output directory exists
mkpath("examples/output")

# Try loading Luxor for SVG export
try
    @eval using Luxor
    plot_circuit(circuit; seed=42)
    println("✓ SVG diagram saved to examples/output/circuit_tutorial.svg")
    println("  You can open this file in a web browser or include it in LaTeX documents")
catch e
    if isa(e, ArgumentError) && occursin("Luxor", string(e))
        println("ℹ Luxor.jl not available - SVG export skipped")
        println("  To enable SVG export, install Luxor: ]add Luxor")
    else
        println("⚠ SVG export failed: $(sprint(showerror, e))")
    end
end
println()

SVG Visualization (optional - requires Luxor.jl):

✓ SVG diagram saved to examples/output/circuit_tutorial.svg
  You can open this file in a web browser or include it in LaTeX documents



In [6]:
list_observables()

2-element Vector{String}:
 "DomainWall"
 "BornProbability"

## Section 5: Observables and Simulating Circuits

Once a circuit is built and visualized, we can execute it using `simulate!()`. The simulation requires:

1. A SimulationState with initialized MPS and RNG registry
2. The circuit to execute
3. Number of trajectory samples (n_circuits)

**TIP**: Use `list_observables()` to discover all available observable types that can be measured during simulation (e.g., DomainWall, Entanglement, etc.)

**CRITICAL**: Using the same RNG seed for visualization and simulation ensures they show identical stochastic branch choices.

In [7]:
println("Simulating circuit (seed=42)...")
println()

# Create simulation state with RNG registry
# Each RNG source (ctrl, proj, haar, born) gets its own deterministic seed
state = SimulationState(
    L = L, 
    bc = bc, 
    rng = RNGRegistry(ctrl=42, proj=1, haar=2, born=3)
)

# Initialize state to product state with x=1/16
# This represents all qubits in a coherent state |ψ⟩ = |+⟩ ⊗ ... ⊗ |+⟩
# where |+⟩ = (|0⟩ + |1⟩)/√2, rotated by angle θ with cos²(θ/2) = 1/16
initialize!(state, ProductState(x0=1//16))

# Execute circuit with one trajectory
simulate!(circuit, state; n_circuits=1)

println("✓ Circuit simulation complete")
println("  Final state prepared with $(n_steps) timesteps")
println("  Stochastic branches sampled with seed=42")
println()

Simulating circuit (seed=42)...

✓ Circuit simulation complete
  Final state prepared with 50 timesteps
  Stochastic branches sampled with seed=42


  Final state prepared with 50 timesteps
  Stochastic branches sampled with seed=42



## Section 6: Comparing Visualization and Simulation

A key feature of the Circuit API is that visualization and simulation are deterministically linked via RNG seeds. The same seed produces:
- Same gate sequence in SVG visualization
- Same quantum trajectory in simulation

This ensures that what you see in the diagram is exactly what gets executed.

In [8]:
println("Verification: Deterministic consistency")
println()

# Run simulation with same seed
state_verify = SimulationState(
    L = L, 
    bc = bc, 
    rng = RNGRegistry(ctrl=42, proj=1, haar=2, born=3)
)
initialize!(state_verify, ProductState(x0=1//16))

# Track an observable to demonstrate observable access later
track!(state_verify, :dw1 => DomainWall(; order=1, i1_fn=() -> 1))
record!(state_verify)

simulate!(circuit, state_verify; n_circuits=1)

println("✓ Simulation with seed=42 complete")
println()
println("ℹ The visualization and simulation used identical RNG branches")
println("  → What you see in the diagram is what gets executed")
println("  → This enables reliable debugging and reproducible results")
println()

Verification: Deterministic consistency

✓ Simulation with seed=42 complete

ℹ The visualization and simulation used identical RNG branches
  → What you see in the diagram is what gets executed
  → This enables reliable debugging and reproducible results



ℹ The visualization and simulation used identical RNG branches
  → What you see in the diagram is what gets executed
  → This enables reliable debugging and reproducible results



## Demo A: Recording at Every Gate

The new `record_when=:every_gate` option records observables after **each individual gate** execution, providing fine-grained tracking of quantum state evolution.

This is useful when you want to see how observables change at the finest time resolution - after every gate application, not just after complete circuit executions.

In [9]:
# Demo A: Track DomainWall at each gate
circuit = Circuit(L=4, bc=:open, n_steps=10) do c
    apply!(c, HaarRandom(), StaircaseRight(1))
    apply!(c, Reset(), SingleSite(2))
end

rng = RNGRegistry(ctrl=42, proj=43, haar=44, born=45)
state = SimulationState(L=4, bc=:open, rng=rng)
initialize!(state, ProductState(x0=1//16))
track!(state, :dw => DomainWall(order=1, i1_fn=() -> 1))

# Record after EVERY gate (fine-grained tracking)
simulate!(circuit, state; n_circuits=5, record_when=:every_gate)

println("Number of recordings: ", length(state.observables[:dw]))
println("Expected: ", 5 * 10 * 2, " gates (5 circuits × 10 steps × 2 ops)")

# Plot shows many data points - evolution at each gate
# plot(state.observables[:dw], title="DomainWall at every gate")

Number of recordings: 100
Expected: 100 gates (5 circuits × 10 steps × 2 ops)


In [11]:
state.observables[:dw]

100-element Vector{Float64}:
 2.5947615063801948
 1.857672491242524
 2.789264210276341
 2.445399380408354
 2.0540530099008807
 2.0540530099008825
 2.727074450358303
 3.635118366448285
 3.805313752107768
 3.610908915934879
 ⋮
 2.8858375770349176
 3.8067678156894096
 3.5452893250035324
 3.5193222153205053
 3.3952588086925046
 3.271000209741456
 3.271000209741452
 2.7903044639412555
 2.008534349281686

## Demo B: Recording Final State Only

In contrast to Demo A, the `record_when=:final_only` option records observables **only once** - after the very last gate of the entire simulation completes.

This is efficient when you only need the final result and don't care about intermediate evolution. Compare the number of recordings:
- Demo A (`:every_gate`): 100 recordings (5 circuits × 10 steps × 2 ops)
- Demo B (`:final_only`): **1 recording** (final state only)

In [12]:
# Demo B: Track DomainWall only at final state
circuit = Circuit(L=4, bc=:open, n_steps=10) do c
    apply!(c, HaarRandom(), StaircaseRight(1))
    apply!(c, Reset(), SingleSite(2))
end

rng = RNGRegistry(ctrl=42, proj=43, haar=44, born=45)
state = SimulationState(L=4, bc=:open, rng=rng)
initialize!(state, ProductState(x0=1//16))
track!(state, :dw => DomainWall(order=1, i1_fn=() -> 1))

# Record ONLY at the very end
simulate!(circuit, state; n_circuits=5, record_when=:final_only)

println("Number of recordings: ", length(state.observables[:dw]))
println("Expected: 1 (final state only)")

# Only 1 data point - the final state
println("Final DomainWall value: ", state.observables[:dw][1])

Number of recordings: 1
Expected: 1 (final state only)
Final DomainWall value: 2.008534349281686


## Section 7: Accessing Recorded Observable Data

After tracking and recording observables during simulation, you can access the measured data through the `state.observables` dictionary using the observable name as the key.

In [13]:
println("Accessing recorded observable data:")
println()

# Access the domain wall observable we tracked
dw_values = state_verify.observables[:dw1]
println("Domain wall measurements: ", dw_values)
println()
println("ℹ Observable data is stored as a vector, with one measurement per timestep")
println("  You can plot, analyze, or export this data for further study")
println()

Accessing recorded observable data:

Domain wall measurements: [1.0, 3.4908224546003344]

ℹ Observable data is stored as a vector, with one measurement per timestep
  You can plot, analyze, or export this data for further study

[1.0, 3.4908224546003344]

ℹ Observable data is stored as a vector, with one measurement per timestep
  You can plot, analyze, or export this data for further study



## Summary

### What You Learned

1. Build circuits with do-block syntax (lazy/symbolic)
2. Visualize circuits with `plot_circuit` (SVG)
3. Simulate circuits with `simulate!()` (deterministic execution)
4. Use RNG seeds to ensure visualization matches simulation
5. Track and record observables during simulation
6. Access recorded observable data via `state.observables`

### Circuit API Workflow

**Build → Visualize → Inspect → Simulate → Analyze**

## Next Steps

Explore these possibilities:

- Experiment with different gate types (Reset, HaarRandom, PauliX, PauliZ)
- Try different geometries (SingleSite, StaircaseRight, StaircaseLeft)
- Vary system size L and timesteps n_steps
- Compare periodic vs open boundary conditions
- Install Luxor.jl for publication-quality SVG diagrams
- Explore different observables with `list_observables()`

---

**Tutorial complete!**